In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn.metrics import root_mean_squared_error,mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

In [ ]:
ari = pd.read_csv("data_ari.csv",sep=",",dtype={'location':str,'year_week':str,
                                                'value':np.float32,'relative_humidity_2m':np.float64,
                                                'temperature_2m_max':np.float64,'temperature_2m_min':np.float64},
                                                parse_dates=['truth_date'])
name_ari = ari['location'].unique()
mape_ari = pd.DataFrame(columns=['location', 'model', 'prediction_window', 'mape', 'nrmse'])

In [ ]:
ili = pd.read_csv("data_ili.csv",sep=",",dtype={'location':str,'year_week':str,
                                                'value':np.float32,'relative_humidity_2m':np.float64,
                                                'temperature_2m_max':np.float64,'temperature_2m_min':np.float64},
                                                parse_dates=['truth_date'])
ili = ili.drop(columns=['Unnamed: 0']).reset_index(drop=True)
name_ili = ili['location'].unique()
mape_ili = pd.DataFrame(columns=['location', 'model', 'prediction_window', 'mape', 'nrmse'])

In [ ]:
summary = ili.groupby("location")["value"].agg(
    mean_incidence="mean",
    median_incidence="median",
    min_incidence="min",
    max_incidence="max"
).reset_index().sort_values("mean_incidence")

print(summary)

In [ ]:
summary = ari.groupby("location")["value"].agg(
    mean_incidence="mean",
    median_incidence="median",
    min_incidence="min",
    max_incidence="max"
).reset_index().sort_values("mean_incidence")

print(summary)

In [ ]:
ili.location.unique()

In [ ]:
ari.location.unique()

In [ ]:
def eval_metrics(y_true, y_pred,alpha=0.2):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # remove nan or not valid values like infinity
    mask = np.isfinite(y_true) & np.isfinite(y_pred)
    if mask.sum() == 0:
        return np.nan, np.nan
    y_clean = y_true[mask]
    y_pred_clean = y_pred[mask]
    if (y_clean == 0).sum() > 0:
        S = np.percentile(np.abs(y_clean),90)
    else:
        S = 0
    epsilon = S * alpha
    print(f"Using epsilon: {epsilon}")
    denom = np.maximum(np.abs(y_clean), float(epsilon))
    mae = np.mean(np.abs((y_clean - y_pred_clean) / (denom))) * 100
    rmse = root_mean_squared_error(y_clean, y_pred_clean)/(y_clean.max() - y_clean.min())
    return mae, rmse

In [ ]:
name_ili

In [ ]:
for i in name_ari:
    print(f'the country running is {i}')
    combined_df = pd.read_csv(f"mixed_xgboost_model_{i}_ARI.csv")
    for h in range(4):
        shifted = combined_df["value"].shift(-h)
        preds = combined_df[f"prediction_{h+1}_weeks"]
        valid_idx = ~shifted.isna()
        y_true = shifted[valid_idx]
        y_pred = preds[valid_idx]
        mape, nrmse = eval_metrics(y_true, y_pred, alpha=0)
        mape_ari = pd.concat([
            mape_ari,
            pd.DataFrame([[i, 'seasonal_naive', f"{h+1}_week", mape, nrmse]],
                            columns=['location', 'model', 'prediction_window', 'mape', 'nrmse'])
        ], ignore_index=True)

In [ ]:
mape_ari

In [ ]:
mape_ari.to_csv("resultados/mape_ari_xgboost.csv", index=False,sep=";", decimal=',')

In [ ]:
for i in name_ili:
    print(f'the country running is {i}')
    combined_df = pd.read_csv(f"mixed_xgboost_model_{i}_ILI.csv")
    for h in range(4):
        shifted = combined_df["value"].shift(-h)
        preds = combined_df[f"prediction_{h+1}_weeks"]
        valid_idx = ~shifted.isna()
        y_true = shifted[valid_idx]
        y_pred = preds[valid_idx]
        mape, nrmse = eval_metrics(y_true, y_pred, alpha=0.2)
        mape_ili = pd.concat([
            mape_ili,
            pd.DataFrame([[i, 'naive', f"{h+1}_week", mape, nrmse]],
                            columns=['location', 'model', 'prediction_window', 'mape', 'nrmse'])
        ], ignore_index=True)

In [ ]:
mape_ili.tail(30)

In [ ]:
mape_ili.to_csv("resultados/mape_ili_xgboost.csv", index=False,sep=";", decimal=',')

In [ ]:
for i in name_ili:
    print(f'the country running is {i}')
    df = pd.read_csv(f'resultados/sarima_xgboost/mixed_xgboost_model_{i}_ILI.csv')
    df.to_csv(f'resultados/sarima_xgboost/mixed_xgboost_model_{i}_ILI.csv', index=False,sep=";", decimal=',')


In [ ]:
for i in name_ari:
    print(f'the country running is {i}')
    be_ari = pd.read_excel('resultados/to_plot/BE.xlsx',sheet_name='Ari')
    be = ari[ari['location'] == i]
    print(be_ari.dtypes)


In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

# inside your loop, right after your three lines:
for i in name_ari:
    print(f'the country running is {i}')
    be_ari = pd.read_excel('resultados/to_plot/BE.xlsx', sheet_name='Ari')  # your line
    be = ari[ari['location'] == i]                                         # your line

    # --- align observed with prediction dates ---
    # Use the prediction dates from be_ari to extract the matching observed values from 'be'
    pred_dates = be_ari[['truth_date']].drop_duplicates().sort_values('truth_date')
    obs_sync = pred_dates.merge(be[['truth_date', 'value']], on='truth_date', how='left')

    # --- build the plot ---
    Path('figs').mkdir(parents=True, exist_ok=True)
    x = pred_dates['truth_date']

    plt.figure(figsize=(10, 5))
    # Observed (from 'be' as you requested)
    plt.plot(x, obs_sync['value'], label='Observed (value)')

    # Predictions (from 'be_ari' as you requested)
    model_cols = [c for c in be_ari.columns if c not in ('truth_date', 'value')]
    for col in model_cols:
        plt.plot(x, be_ari[col], label=col)

    plt.title(f"{i} – ARI – Observed vs model forecasts")
    plt.xlabel("Date")
    plt.ylabel("Incidence")
    plt.legend(loc='best', ncol=2, fontsize=8)
    plt.tight_layout()

    out_png = f"figs/{i}_ARI_models.png"
    plt.savefig(out_png, dpi=300)
    plt.close()
    print(f"Saved: {out_png}")


In [ ]:
RO = ili[ili['location'] == 'RO']
RO[RO['truth_date']>'2023-10-01']

In [ ]:
# inside your loop, right after your three lines:
import matplotlib.cm as cm
for i in ['BE','LU','RO']:
    print(f'the country running is {i}')
    be_ari = pd.read_excel(f'resultados/to_plot/{i}.xlsx', sheet_name='Ari')  # your line
    # --- build the plot ---
    x = be_ari['truth_date']

    plt.figure(figsize=(10, 5))
    # Observed (from 'be' as you requested)
    plt.plot(x,be_ari['value'], label='Observed (value)',color='black')
    
    # Predictions (from 'be_ari' as you requested)
    model_cols = [c for c in be_ari.columns if c not in ('truth_date', 'value')]
    colors = cm.get_cmap('tab20', len(model_cols)) 
    for j,col in enumerate(model_cols):
        plt.plot(x, be_ari[col], label=col,color=colors(j))

    plt.title(f"{i} – ARI – Observed vs model forecasts")
    plt.xlabel("Date")
    plt.ylabel("Incidence")
    plt.legend(loc='best', ncol=2, fontsize=8)
    plt.tight_layout()

    out_png = f"figs/{i}_ARI_models.png"
    plt.savefig(out_png, dpi=300)
    plt.close()
    print(f"Saved: {out_png}")


In [ ]:
be_ari.columns

In [ ]:
def plot_train_test(train, test, model_name, location,folder):
    plt.figure(figsize=(15, 7))

    # Plot actual data
    plt.plot(train.index, train["value"], color='blue', label='Training Data')
    plt.plot(test.index, test["value"], color='orange', label='Actual Test Data')

    # Unified forecast column names
    forecast_horizons = {
    'randomforest':('green'),
    'arimax':('red'),
    'sarima_randomforest':('purple'),
    'sarima_xgboost':('brown'),
    'rnn':('magenta'),
    'arima':('pink'),
    'sarimax':('gray'),
    'sarima':('olive'),
    'xgboost':('black'),
    'naive':('yellow'),
    'seasonal_naive':('cyan')
    }

    for col, (color) in forecast_horizons.items():
        if col in test.columns:
            # Shift predictions forward by their horizon
            plt.plot(test.index, test[col], linestyle='--', color=color, label=f'{model_name} {col}')

    plt.legend(loc='upper left')
    plt.title(f"{model_name} Forecasting – {location}")
    plt.xlabel("Date")
    plt.ylabel("Value")
    plt.grid(True)
    plt.tight_layout()
    

In [ ]:
plot_train_test(be,be_ari,'Mixed XGBoost','Belgium','figs')

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

def plot_train_test_simple(be, be_ari, location, folder="figs"):
    # Ensure datetime index
    be = be.sort_values("truth_date").set_index("truth_date")
    be_ari = be_ari.sort_values("truth_date").set_index("truth_date")

    plt.figure(figsize=(15, 7))

    # Observed: full series (blue) + test observed (orange from be_ari dates)
    plt.plot(be.index, be["value"], color='blue', label='Training Data')
    if "value" in be_ari.columns:
        plt.plot(be_ari.index, be_ari["value"], color='orange', label='Actual Test Data')

    # Exact Excel column names → colors (simple, no normalization)
    forecast_colors = {
        'Random Forest':        'green',
        'Arimax':               'red',
        'Sarima/RandomForest':  'purple',
        'Sarima/Xgboost':       'brown',
        'RNN':                  'magenta',
        'Arima':                'pink',
        'Sarimax':              'gray',
        'Sarima':               'olive',
        'XGboost':              'black',
        'Naive':                'yellow',
        'Seasonal_Naive':       'cyan',
    }

    # Plot predictions if present
    for col, color in forecast_colors.items():
        if col in be_ari.columns:
            plt.plot(be_ari.index, be_ari[col], linestyle='--', color=color, label=col)

    plt.legend(loc='upper left', ncol=2, fontsize=9)
    plt.title(f"ARI Forecasting – {location}")
    plt.xlabel("Date")
    plt.ylabel("Value")
    plt.grid(True)
    plt.tight_layout()

    Path(folder).mkdir(parents=True, exist_ok=True)
    out_png = f"{folder}/{location}_ARI_models.png"
    plt.savefig(out_png, dpi=300)
    plt.close()
    print(f"Saved: {out_png}")


In [ ]:
for i in name_ari:
    print(f'the country running is {i}')
    be_ari = pd.read_excel(f'resultados/to_plot/{i}.xlsx', sheet_name='Ari')
    be = ari[ari['location'] == i].copy()
    plot_train_test_simple(be, be_ari, i, folder="figs")


In [ ]:
import joblib
import glob

all_importances = []

# adjust the path pattern to your filenames
for file in glob.glob("models_rf/rf_model_*_h*.joblib"):
    rf = joblib.load(file)

    # extract metadata from filename
    parts = file.split("_")
    country = parts[2]  # adapt depending on your naming scheme
    horizon = parts[3].replace("h", "").replace(".joblib","")

    # collect importances
    importances = rf.feature_importances_
    df = pd.DataFrame({
        "feature": rf.feature_names_in_,
        "importance": importances,
        "country": country,
        "horizon": int(horizon)
    })
    all_importances.append(df)

all_importances = pd.concat(all_importances, ignore_index=True)
